In [15]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.charts import Histogram, Donut
from bokeh.io import output_notebook
from bokeh.models.formatters import DatetimeTickFormatter as dttf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

In [16]:
output_notebook() #loading bokeh for notebook output, will change to file for later dashboard
# output_file("test.html", title = 'test for output')

Loading BokehJS ...

In [17]:
df=pd.read_json('teamsdone.json')

In [18]:
#gathers the total number of SC2 Tournaments Played
df['totalsc2'] = df.groupby('PlayerId')['PlayerId'].transform('count')

In [19]:
#popped ('DOB' Column for now, as it returned improper values)
df.DOB = pd.to_datetime(df.DOB)
df.EndDate = pd.to_datetime(df.EndDate)

In [20]:
df.Prize_USD = df.Prize_USD / df.TeamPlayers
df.Prize_USD = (df.Prize_USD.astype(int))
df[df.TotalUSDPrize < df.Prize_USD]
df['date']=df['EndDate']
df.index = df['date']
df.sort_index()

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,...,Prize,Prize_USD,RankText,TeamPlayers,Team_On,TotalUSDPrize,TournamentName,team_history,totalsc2,date
date,,,,,,,,,,,,,,,,,,,,,
2010-02-28,23.0,ua,EUR,DIMAGA,1986-06-03,2010-02-28,1.361800,151,Dmytro,Filipchuk,...,100.0,136,WIN,1,Unaffiliated,66288.11,ZOTAC Cup Starcraft II #1 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-02-28
2010-03-07,22.0,be,EUR,Orly,1987-06-19,2010-03-07,1.362350,151,Olivier,Bellemans,...,100.0,136,WIN,1,Unaffiliated,2596.46,ZOTAC Cup Starcraft II #2 (EU),"[{'team': 'LowLandLions', 'history': [12681792...",6,2010-03-07
2010-03-14,23.0,ua,EUR,DIMAGA,1986-06-03,2010-03-14,1.376900,151,Dmytro,Filipchuk,...,100.0,137,WIN,1,Unaffiliated,66288.11,ZOTAC Cup Starcraft II #3 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-03-14
2010-03-21,20.0,gb,EUR,DeMusliM,1990-01-24,2010-03-21,1.353400,151,Benjamin,Baker,...,100.0,135,WIN,1,Unaffiliated,31612.56,ZOTAC Cup Starcraft II #4 (EU),"[{'team': 'EURONICS Gaming', 'history': [14542...",47,2010-03-21
2010-03-28,26.0,se,EUR,HayprO,1983-06-19,2010-03-28,1.342230,151,Hayder,Hussein,...,100.0,134,WIN,1,Unaffiliated,7966.80,ZOTAC Cup Starcraft II #5 (EU),"[{'team': 'Team Liquid', 'history': [127802880...",10,2010-03-28
2010-04-01,NaN,es,EUR,LucifroN,NaT,2010-04-01,1.357183,151,Pedro,Moreno Durán,...,500.0,678,WIN,1,1,62777.29,Go4SC2 March 2010 Monthly Finals,None,46,2010-04-01
2010-04-03,NaN,de,EUR,Mardow,NaT,2010-04-03,1.343287,151,Sebastian,Dutsch,...,100.0,134,WIN,1,Unaffiliated,134.33,Go4SC2 EU #3,[],1,2010-04-03
2010-04-04,24.0,de,EUR,GoOdy,1985-10-27,2010-04-04,1.350200,151,Sascha,Lupp,...,100.0,135,WIN,1,Unaffiliated,29551.33,ZOTAC Cup Starcraft II #6 (EU),"[{'team': 'ESC Gaming', 'history': [1351123200...",144,2010-04-04
2010-04-10,29.0,ua,EUR,White-Ra,1980-11-15,2010-04-10,1.343287,151,Aleksey,Krupnyk,...,100.0,134,WIN,1,mousesports,50614.31,Go4SC2 EU #5,"[{'team': 'mousesports', 'history': [126938880...",42,2010-04-10


In [21]:
dfid = df[df['PlayerId']==1058]
dfid.index = dfid['date']
dfid = dfid.sort_index()
dfid['CumulativePrizeToDate']=dfid.Prize_USD.cumsum()
dfid['PrizeToDate'] = dfid.CumulativePrizeToDate - dfid.Prize_USD
dfid['WinsToDate']=np.arange(len(dfid))
dfid['AverageWinsToDate']=np.where(dfid['WinsToDate']==0, 0, dfid.PrizeToDate/dfid.WinsToDate)

In [22]:
pid = df['PlayerId'].unique()
for i in pid:
    if i != 1058:
        dfi = df[df['PlayerId']==i]
        dfi.index = dfi['date']
        dfi = dfi.sort_index()
        dfi['CumulativePrizeToDate']=dfi.Prize_USD.cumsum()
        dfi['PrizeToDate'] = dfi.CumulativePrizeToDate - dfi.Prize_USD
        dfi['WinsToDate']=np.arange(len(dfi))
        dfi['AverageWinsToDate']=np.where(dfi['WinsToDate']==0, 0, dfi.PrizeToDate/dfi.WinsToDate)
        dfid = dfid.append(dfi)

In [23]:
dfid

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,...,Team_On,TotalUSDPrize,TournamentName,team_history,totalsc2,date,CumulativePrizeToDate,PrizeToDate,WinsToDate,AverageWinsToDate
date,,,,,,,,,,,,,,,,,,,,,
2010-02-28,23.0,ua,EUR,DIMAGA,1986-06-03,2010-02-28,1.361800,151,Dmytro,Filipchuk,...,Unaffiliated,66288.11,ZOTAC Cup Starcraft II #1 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-02-28,136,0,0,0.000000
2010-03-14,23.0,ua,EUR,DIMAGA,1986-06-03,2010-03-14,1.376900,151,Dmytro,Filipchuk,...,Unaffiliated,66288.11,ZOTAC Cup Starcraft II #3 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-03-14,273,136,1,136.000000
2010-08-15,24.0,ua,EUR,DIMAGA,1986-06-03,2010-08-15,1.289676,151,Dmytro,Filipchuk,...,mTw,66288.11,Go4SC2 EU #29,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-08-15,530,273,2,136.500000
2010-08-22,24.0,ua,USD,DIMAGA,1986-06-03,2010-08-22,1.000000,151,Dmytro,Filipchuk,...,mTw,66288.11,IEM V - Cologne (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-08-22,1930,530,3,176.666667
2010-10-03,24.0,ua,EUR,DIMAGA,1986-06-03,2010-10-03,1.379100,151,Dmytro,Filipchuk,...,mTw,66288.11,ZOTAC Cup Starcraft II #25 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-03,2067,1930,4,482.500000
2010-10-17,24.0,ua,EUR,DIMAGA,1986-06-03,2010-10-17,1.398050,151,Dmytro,Filipchuk,...,mTw,66288.11,ZOTAC Cup Starcraft II #27 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-17,2206,2067,5,413.400000
2010-10-23,24.0,ua,USD,DIMAGA,1986-06-03,2010-10-23,1.000000,151,Dmytro,Filipchuk,...,mTw,66288.11,BlizzCon 2010 (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-23,5206,2206,6,367.666667
2010-11-10,24.0,ua,EUR,DIMAGA,1986-06-03,2010-11-10,1.372742,151,Dmytro,Filipchuk,...,mTw,66288.11,Cyborg Cup #1-3,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-10,5480,5206,7,743.714286
2010-11-24,24.0,ua,EUR,DIMAGA,1986-06-03,2010-11-24,1.335140,151,Dmytro,Filipchuk,...,mTw,66288.11,Cyborg Cup #1-5,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-24,5747,5480,8,685.000000


In [24]:
df = dfid
# # #checks for playerIds of players who had a team affiliation:
# pid = df[df['Team_On']!='Unaffiliated']['PlayerId'].unique()
# pid

In [25]:
# dfid = df[df['PlayerId']==1058]
# dfid = dfid.sort_index()
# hist = list(dfid['team_history'])
# b = dfid['date']
# l = []
# for h in range(len(hist)):
#     i=1
#     for j in range(len(hist[h])):
#         dc = hist[h][j]['history'][1]
# #         print(np.datetime64(dc,'ms'), b[h], i)
#         if(b[h] > np.datetime64(dc,'ms')):
#             i+=1
# #         print(hist[h][j])
# #         print(b[h])
# #         if b[h] < (np.datetime64((hist[h][it]['history'][1]),'ms')):
# # #             i+=1
#     l.append(i)
# dfid['num_teams'] = l
# #iterating
# for i in pid:
#     dfi = df[df['PlayerId']==i]
#     dfi = dfid.sort_index()
#     hist = list(dfi['team_history'])
#     b = dfi['date']
#     q = []
#     for h in range(len(hist)):
#         i=1
#         for j in range(len(hist[h])):
#             dc = hist[h][j]['history'][1]
#     #         print(np.datetime64(dc,'ms'), b[h], i)
#             if(b[h] > np.datetime64(dc,'ms')):
#                 i+=1
#         q.append(i)
#     dfi['num_teams'] = q
#     dfid = dfid.append(dfi)

In [26]:
dft = dfid
dft.describe()

,Age_at_win,ExchangeRate,GameId,PlayerId,Prize,Prize_USD,TeamPlayers,TotalUSDPrize,totalsc2,CumulativePrizeToDate,PrizeToDate,WinsToDate,AverageWinsToDate
count,14248.000000,19969.000000,19969.0,19969.000000,1.996900e+04,19969.000000,19969.000000,19969.000000,19969.000000,19969.000000,19969.000000,19969.000000,19969.000000
mean,21.834924,0.900765,151.0,4338.052331,8.958006e+05,1185.711653,1.665532,94423.296888,71.627673,48506.761881,47321.050228,35.313836,1169.097463
std,2.998722,7.955835,0.0,6066.001574,4.934940e+06,4445.845763,2.157754,116138.686185,72.333421,76584.040015,75862.746578,46.599602,1869.827001
min,13.000000,0.000822,151.0,1000.000000,0.000000e+00,0.000000,1.000000,2.370000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,0.171842,151.0,1133.000000,1.000000e+02,75.000000,1.000000,10031.990000,20.000000,2192.000000,1627.000000,5.000000,227.142857
50%,22.000000,1.000000,151.0,1507.000000,4.500000e+02,250.000000,1.000000,44878.650000,55.000000,15320.000000,14228.000000,19.000000,670.166667
75%,24.000000,1.000000,151.0,3824.000000,5.000000e+03,908.000000,1.000000,134137.430000,98.000000,60613.000000,58920.000000,48.000000,1351.553846
max,34.000000,648.792620,151.0,40763.000000,1.000000e+08,200000.000000,19.000000,506645.420000,365.000000,506625.000000,503911.000000,364.000000,88487.000000


In [27]:
df = df.set_index(np.arange(len(df))) #changing the index to something nice

In [28]:
df['num_teams'] = np.where( df['team_history']=='Unaffiliated', 0, (df.team_history.str.len()))

In [29]:
df['num_teams'] = df['num_teams'].fillna(0)

In [30]:
p = Histogram(df['Age_at_win'].dropna(), title = 'Age of players that cash in tournaments')

In [31]:
show(p)

In [32]:
p = Histogram(df['totalsc2'].unique(), title='Total Number of Cashes Per Player')

In [33]:
show(p)

In [34]:
df

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,...,TotalUSDPrize,TournamentName,team_history,totalsc2,date,CumulativePrizeToDate,PrizeToDate,WinsToDate,AverageWinsToDate,num_teams
0,23.0,ua,EUR,DIMAGA,1986-06-03,2010-02-28,1.361800,151,Dmytro,Filipchuk,...,66288.11,ZOTAC Cup Starcraft II #1 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-02-28,136,0,0,0.000000,5.0
1,23.0,ua,EUR,DIMAGA,1986-06-03,2010-03-14,1.376900,151,Dmytro,Filipchuk,...,66288.11,ZOTAC Cup Starcraft II #3 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-03-14,273,136,1,136.000000,5.0
2,24.0,ua,EUR,DIMAGA,1986-06-03,2010-08-15,1.289676,151,Dmytro,Filipchuk,...,66288.11,Go4SC2 EU #29,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-08-15,530,273,2,136.500000,5.0
3,24.0,ua,USD,DIMAGA,1986-06-03,2010-08-22,1.000000,151,Dmytro,Filipchuk,...,66288.11,IEM V - Cologne (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-08-22,1930,530,3,176.666667,5.0
4,24.0,ua,EUR,DIMAGA,1986-06-03,2010-10-03,1.379100,151,Dmytro,Filipchuk,...,66288.11,ZOTAC Cup Starcraft II #25 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-03,2067,1930,4,482.500000,5.0
5,24.0,ua,EUR,DIMAGA,1986-06-03,2010-10-17,1.398050,151,Dmytro,Filipchuk,...,66288.11,ZOTAC Cup Starcraft II #27 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-17,2206,2067,5,413.400000,5.0
6,24.0,ua,USD,DIMAGA,1986-06-03,2010-10-23,1.000000,151,Dmytro,Filipchuk,...,66288.11,BlizzCon 2010 (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-23,5206,2206,6,367.666667,5.0
7,24.0,ua,EUR,DIMAGA,1986-06-03,2010-11-10,1.372742,151,Dmytro,Filipchuk,...,66288.11,Cyborg Cup #1-3,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-10,5480,5206,7,743.714286,5.0
8,24.0,ua,EUR,DIMAGA,1986-06-03,2010-11-24,1.335140,151,Dmytro,Filipchuk,...,66288.11,Cyborg Cup #1-5,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-24,5747,5480,8,685.000000,5.0
9,24.0,ua,RUB,DIMAGA,1986-06-03,2010-11-28,0.031859,151,Dmytro,Filipchuk,...,66288.11,ASUS Autumn 2010 (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-28,6384,5747,9,638.555556,5.0


In [35]:
#AveragePricePerPlayer a lot trickier now. Will redo graph later

In [36]:
# p = Histogram(np.log(df[df['AveragePrizePerPlayer']>0]['AveragePrizePerPlayer'].unique()), title='Log Average Prize Per Player')

In [37]:
# show(p)

In [38]:
p = Donut(df['CountryCode'], title = "Country of Origin")
show(p)

In [39]:
df['country'] = 'Other'
top_countries = ['kr','fr','de','cn','us','ua','se','ru','pl']
for a in top_countries:
    df['country']= np.where(df['CountryCode']==a, a, df['country'])

p = Donut(df['country'], title = "Country of Origin (top 10)")
show(p)

In [40]:
p = Donut(df['CurrencyCode'], title = 'Currency of Prize Money')
show(p)

In [41]:
df['currency'] = 'Other'
top_countries = ['USD', 'KRW', 'EUR', 'CNY', 'AUD']
for a in top_countries:
    df['currency']= np.where(df['CurrencyCode']==a, a, df['currency'])

p = Donut(df['currency'], title = "Currency (Top 5)")

show(p)

In [42]:
pd.set_option('max_columns', 50)

In [43]:
df

,Age_at_win,CountryCode,CurrencyCode,CurrentHandle,DOB,EndDate,ExchangeRate,GameId,NameFirst,NameLast,Note,PlayerId,Prize,Prize_USD,RankText,TeamPlayers,Team_On,TotalUSDPrize,TournamentName,team_history,totalsc2,date,CumulativePrizeToDate,PrizeToDate,WinsToDate,AverageWinsToDate,num_teams,country,currency
0,23.0,ua,EUR,DIMAGA,1986-06-03,2010-02-28,1.361800,151,Dmytro,Filipchuk,None,1058,100.0,136,WIN,1,Unaffiliated,66288.11,ZOTAC Cup Starcraft II #1 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-02-28,136,0,0,0.000000,5.0,ua,EUR
1,23.0,ua,EUR,DIMAGA,1986-06-03,2010-03-14,1.376900,151,Dmytro,Filipchuk,None,1058,100.0,137,WIN,1,Unaffiliated,66288.11,ZOTAC Cup Starcraft II #3 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-03-14,273,136,1,136.000000,5.0,ua,EUR
2,24.0,ua,EUR,DIMAGA,1986-06-03,2010-08-15,1.289676,151,Dmytro,Filipchuk,None,1058,200.0,257,WIN,1,mTw,66288.11,Go4SC2 EU #29,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-08-15,530,273,2,136.500000,5.0,ua,EUR
3,24.0,ua,USD,DIMAGA,1986-06-03,2010-08-22,1.000000,151,Dmytro,Filipchuk,None,1058,1400.0,1400,3rd,1,mTw,66288.11,IEM V - Cologne (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-08-22,1930,530,3,176.666667,5.0,ua,USD
4,24.0,ua,EUR,DIMAGA,1986-06-03,2010-10-03,1.379100,151,Dmytro,Filipchuk,None,1058,100.0,137,WIN,1,mTw,66288.11,ZOTAC Cup Starcraft II #25 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-03,2067,1930,4,482.500000,5.0,ua,EUR
5,24.0,ua,EUR,DIMAGA,1986-06-03,2010-10-17,1.398050,151,Dmytro,Filipchuk,None,1058,100.0,139,WIN,1,mTw,66288.11,ZOTAC Cup Starcraft II #27 (EU),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-17,2206,2067,5,413.400000,5.0,ua,EUR
6,24.0,ua,USD,DIMAGA,1986-06-03,2010-10-23,1.000000,151,Dmytro,Filipchuk,None,1058,3000.0,3000,4th,1,mTw,66288.11,BlizzCon 2010 (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-10-23,5206,2206,6,367.666667,5.0,ua,USD
7,24.0,ua,EUR,DIMAGA,1986-06-03,2010-11-10,1.372742,151,Dmytro,Filipchuk,None,1058,200.0,274,WIN,1,mTw,66288.11,Cyborg Cup #1-3,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-10,5480,5206,7,743.714286,5.0,ua,EUR
8,24.0,ua,EUR,DIMAGA,1986-06-03,2010-11-24,1.335140,151,Dmytro,Filipchuk,None,1058,200.0,267,WIN,1,mTw,66288.11,Cyborg Cup #1-5,"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-24,5747,5480,8,685.000000,5.0,ua,EUR
9,24.0,ua,RUB,DIMAGA,1986-06-03,2010-11-28,0.031859,151,Dmytro,Filipchuk,None,1058,20000.0,637,1st,1,mTw,66288.11,ASUS Autumn 2010 (StarCraft II),"[{'team': 'IsIMBA', 'history': [1461024000000,...",85,2010-11-28,6384,5747,9,638.555556,5.0,ua,Other


In [44]:
# p = figure()
# p.circle(x = df['EndDate'], y =df['Prize_USD'])
# p.xaxis.formatter = dttf.months

In [45]:
# show(p) #need to figure out date/Time on Bokeh

In [46]:
df['Team_On_Backup']=df['Team_On']

In [47]:
df['Team_On']=np.where(df['Team_On']==1, "Unaffiliated", df['Team_On']) #cleaning

In [48]:
p = Donut(df['Team_On'], title = 'Distribution of Winners')
show(p)

In [49]:
a = df.Team_On.value_counts().nlargest(26)
top_teams = list(a.index)
df['teams'] = 'Other'
for a in top_teams:
    df['teams']= np.where(df['Team_On']==a, a, df['teams'])

p = Donut(df['teams'], title = "Teams (Top 25)")
show(p)

In [50]:
df['RankText'].unique()

array(['WIN', '3rd', '4th', '1st', '5th-8th', '2nd', '3rd-4th', '9th-16th',
       '7th-8th', 'LOSE', '17th-20th', 'See Note', '13th-16th',
       '17th-24th', '33rd-64th', '17th-32nd', '5th-12th', '6th', 'Code S',
       '9th-12th', 'Round 3', '25th-32nd', 'Round 2', '25th-48th',
       '5th-6th', '5th', '9th-11th', 'Round 1', '11th-14th', '7th',
       '6th-8th', '2 Wins', '8 Wins', '9th-10th', '5 Wins', '1 Win', '-',
       '9th', '8th', '31st-60th', '7th-12th', '29th-36th', '3 Wins',
       '21st-24th', '1st-8th', '11th-12th', '13th', '15th-18th',
       '19th-26th', '27th-32nd', '10th-12th', '10th-11th', '7th-10th',
       '13th-14th', '37th-44th', '11th', '25th-28th', '10th',
       'DISQUALIFIED', '12th', '14th', '15th', '1st-4th', '4 Wins',
       '7 Wins'], dtype=object)

In [51]:
#Changing Ranks to Categoricals
df['Rank'] = 'Cash Only'

win = ['WIN','1st','1st-4th']
for a in win:
    df['Rank']=np.where(df['RankText'] == a, 'Winners', df['Rank'])
    
second = ['2nd', '7 Wins','8 Wins','6 Wins', '3rd']
for a in second:
    df['Rank']=np.where(df['RankText'] == a, 'Runners Up', df['Rank'])

In [52]:
p = Donut(df['Rank'], title = 'Distribution of Winners')
show(p)

In [54]:
df_extra = pd.read_csv('extra_features.json')
df = df.merge(df_extra, on=['date','PlayerId'])

KeyboardInterrupt: 

In [ ]:
df_clean = df
df_messy = df
df_messy.to_json('messy.json')

In [ ]:
df_clean

In [ ]:
to_drop = ['CountryCode','CurrencyCode','CurrentHandle','ExchangeRate','GameId','NameFirst','NameLast','Note',
            'PlayerId','Prize','RankText','TeamPlayers','Team_On','TournamentName','team_history',
            'Team_On_Backup','DOB','EndDate','TotalUSDPrize', 'Prize_USD', 'CumulativePrizeToDate',
            'AverageWinsToDate','num_teams','date','totalsc2']
df_dropped = df_clean.drop(to_drop, axis=1)

In [ ]:
df_dropped.head()

In [ ]:
df_dums = pd.get_dummies(data=df_dropped, columns = ['Rank','currency','teams','country'])
df_dums['Age_at_win'] = df_dums['Age_at_win'].fillna(df_dums['Age_at_win'].mean())

In [ ]:
df_dums.info()

In [ ]:
#prep for classification
df_class = df_dums.drop(['Rank_Runners Up','Rank_Cash Only'], axis=1)
#test to see if there is leakage in the average/total:
y = df_class.pop('Rank_Winners')
df_class.head()

In [ ]:
X, X_finaltest, y, y_final_test = train_test_split(df_class,y, random_state=314)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
def standard_confusion_matrix(y_true, y_pred):
    """Make confusion matrix with format:
                  -----------
                  | TP | FP |
                  -----------
                  | FN | TN |
                  -----------
    Parameters
    ----------
    y_true : ndarray - 1D
    y_pred : ndarray - 1D

    Returns
    -------
    ndarray - 2D
    """
    [[tn, fp], [fn, tp]] = confusion_matrix(y_true, y_pred)
    return np.array([[tp, fp], [fn, tn]])

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

#Cross-Validation
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Train Crossval Accuracy: " + str(scores))
print("Predicted Accuracy: "+ str(sum(scores)/5))
preds = clf.predict(X_test)
score = clf.score(X_test,y_test)
print('Test Accuracy: ' + str(score))
print(standard_confusion_matrix(y_test,preds))

Baseline:
Train Crossval Accuracy: [ 0.73386738  0.73030708  0.73030708  0.72796082  0.73496659]
Predicted Accuracy: 0.731481788518
Test Accuracy: 0.727297008547
[[ 125   92]
 [ 929 2598]]

Train Crossval Accuracy: [ 0.72051624  0.71918113  0.71428571  0.71460374  0.72071269]
Predicted Accuracy: 0.717859904684
Test Accuracy: 0.717414529915
[[  14   18]
 [1040 2672]]

Average:
Train Crossval Accuracy: [ 0.73386738  0.73119715  0.72808189  0.730187    0.73674833]
Predicted Accuracy: 0.732016349235
Test Accuracy: 0.727564102564
[[ 136  102]
 [ 918 2588]]

In [ ]:
clf = GradientBoostingClassifier(learning_rate=.01, max_features=10,n_estimators=1000)
clf.fit(X_train, y_train)

#Cross-Validation
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Train Crossval Accuracy: " + str(scores))
print("Predicted Accuracy: "+ str(sum(scores)/5))

preds = clf.predict(X_test)
score = clf.score(X_test,y_test)
print('Test Accuracy: ' + str(score))
print(standard_confusion_matrix(y_test,preds))

Baseline Test (No Engineered Features):
Train Crossval Accuracy: [ 0.73965287  0.73742768  0.7329773   0.73063224  0.73853007]
Predicted Accuracy: 0.735844031363
Test Accuracy: 0.729166666667
[[ 130   90]
 [ 924 2600]]

Train Crossval Accuracy: [ 0.73920783  0.73742768  0.73164219  0.730187    0.7403118 ]
Predicted Accuracy: 0.735755301345
Test Accuracy: 0.729433760684
[[ 127   86]
 [ 927 2604]]

Test with AverageToDate:
Train Crossval Accuracy: [ 0.73787272  0.73876279  0.73119715  0.73152271  0.74253898]
Predicted Accuracy: 0.736378869662
Test Accuracy: 0.732905982906
[[ 143   89]
 [ 911 2601]]

In [ ]:
chnce = 1 - 5574/(5574+4406+9989)
print('Random Chance: ' + str(chnce))